In [17]:
import psycopg2
import pandas as pd

import re
from ipywidgets import interact, interactive, Layout, interactive_output,VBox
import ipywidgets as widgets
from IPython.display import display, HTML
from ipywidgets import HBox, Label, FloatSlider,Dropdown,IntSlider,RadioButtons, Box, Layout
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib


from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
import pickle

In [18]:
conn = psycopg2.connect( 
                        host = 'project.cgxhdwn5zb5t.us-east-1.rds.amazonaws.com',
                        port = 5432, 
                        user = 'postgres',
                        password = 'Admin123',
                        database = 'postgres')
cursor = conn.cursor()

In [19]:
DEC2FLOAT = psycopg2.extensions.new_type(
    psycopg2.extensions.DECIMAL.values,
    'DEC2FLOAT',
    lambda value, curs: float(value) if value is not None else None)
psycopg2.extensions.register_type(DEC2FLOAT)

In [20]:
cursor.execute('Select * from "ahshouseholdclass2"')
rows = cursor.fetchall()
col_names = []
for elt in cursor.description:
    col_names.append(elt[0])

df = pd.DataFrame(data=rows, columns=col_names )

First, import our data and our prebuilt model.

In [21]:
predictors = list(df[['HHAGE', 'HINCP_BIN', 'UTILAMT','UNITSIZE','NUMPEOPLE','BEDROOMS', 
        'PERPOVLVL',  'ELECAMT', 'GASAMT', 
        'TRASHAMT', 'WATERAMT', 'OMB13CBSA','STORIES', 'HHNATVTY'
       ]])

loaded_model = pickle.load(open('test_model.sav', 'rb'))

Next, establish the variables and functions required for our visualization.

In [22]:
IV=['HHAGE', 'UTILAMT','NUMPEOPLE','BEDROOMS']
CV=['HINCP_BIN', 'OMB13CBSA','UNITSIZE' ]

In [36]:
# load the model from pickle and predict according to user-input
def load_model(age, utilamt, income, unitsize, number_of_ppl, bedrooms, region):

    #set the avg on non-widget inputs
    User_input=df[predictors].iloc[0]
    User_input['PERPOVLVL']= df['PERPOVLVL'].mean()
    User_input['ELECAMT']= df['ELECAMT'].mean()
    User_input['GASAMT']= df['GASAMT'].mean()
    User_input['TRASHAMT']= df['TRASHAMT'].mean()
    User_input['WATERAMT']= df['WATERAMT'].mean()
    #User_input['OMB13CBSA']= df['OMB13CBSA'].mean()
    User_input['STORIES']= df['STORIES'].mean()
    User_input['HHNATVTY']= df['HHNATVTY'].mean()
    
    
    #interval variables
    User_input['HHAGE'] = age
    User_input['UTILAMT']= utilamt
    User_input['NUMPEOPLE']= number_of_ppl
    User_input['BEDROOMS']= bedrooms

    #categorical variables
    User_input[income]=1
    User_input[region]=1
    User_input[unitsize]=1

    user_proba=loaded_model.predict_proba([User_input])[:,1]
     
    #show the histgram
    plt.figure(figsize=(15,5))
    plt.yticks([])
    plt.ylabel("Frequency of Predicted Instances")
    plt.hist(df_test[df_test['shock']==0]["probability of income shock"],bins, alpha=.9, label='No Shock',color="#259433")  
    plt.hist(df_test[df_test['shock']==1]["probability of income shock"],bins,alpha=0.7, label='Shock', color="#ff4940")
    plt.axvline(user_proba,0,3000, linewidth=1)
    plt.xticks([user_proba,user_proba])
    plt.legend()
    plt.show()

    return

style={'description_width' :"initial"}
age=widgets.IntSlider(value=30, min=18,max=58,description='Age', layout=Layout(flex='1 1 auto', width='auto'),style=style, continuous_update=False)
utilamt=widgets.IntSlider(value=250, min=0,max=20000,description='Utility Amt',style=style, continuous_update=False)
number_of_ppl=widgets.IntSlider(value=1, min=1,max=20,description='Number of People',style=style, continuous_update=False)
bedrooms=widgets.IntSlider(value=0, min=0,max=10, layout=Layout(flex='1 1 auto', width='auto'),description='Bedrooms',style=style, continuous_update=False)

unitsize=widgets.Dropdown(value='UNITSIZE',style=style, description="Unit Size",disabled=False,layout=Layout(flex='1 1 auto', width='auto'),
                       options=[('Less than 500 square feet','UNITSIZE'),
                               ('500 to 749 square feet','UNITSIZE'),
                              ('750 to 999 square feet','UNITSIZE'),
                              ('1,000 to 1,499 square feet','UNITSIZE'),
                              ('1,500 to 1,999 square feet','UNITSIZE'),
                              ('2,000 to 2,499 square feet','UNITSIZE'),
                              ('2,500 to 2,999 square feet','UNITSIZE'),
                              ('3,000 to 3,999 square feet','UNITSIZE'),
                              ('4,000 square feet or more','UNITSIZE')])

region=widgets.Dropdown(value='OMB13CBSA',style=style, description="Region",disabled=False,
                    options=[("Atlanta-Sandy Springs-Roswell, GA",'OMB13CBSA'),
                            ("Baltimore-Columbia-Towson, MD",'OMB13CBSA'),
                            ('Birmingham-Hoover, AL','OMB13CBSA'),
                            ('Boston-Cambridge-Newton, MA-NH','OMB13CBSA'),
                            ('Chicago-Naperville-Elgin, IL-IN-WI','OMB13CBSA'),
                            ('Cincinnati, OH-KY-IN','OMB13CBSA'),
                            ('Cleveland-Elyria, OH','OMB13CBSA'),
                            ('Dallas-Fort Worth-Arlington, TX', 'OMB13CBSA'),
                            ('Denver-Aurora-Lakewood, CO','OMB13CBSA'),
                            ('Detroit-Warren-Dearborn, MI','OMB13CBSA'),
                            ('Houston-The Woodlands-Sugar Land, TX','OMB13CBSA'),
                            ('Kansas City, MO-KS','OMB13CBSA'),
                            ('Las Vegas-Henderson-Paradise, NV','OMB13CBSA'),
                            ('Los Angeles-Long Beach-Anaheim, CA','OMB13CBSA'),
                            ('Memphis, TN-MS-AR','OMB13CBSA'),
                            ('Miami-Fort Lauderdale-West Palm Beach, FL','OMB13CBSA'),
                            ('Milwaukee-Waukesha-West Allis, WI','OMB13CBSA'),
                            ('Minneapolis-St. Paul-Bloomington, MN-WI','OMB13CBSA'),
                            ('New Orleans-Metairie, LA','OMB13CBSA'),
                            ('New York-Newark-Jersey City, NY-NJ-PA','OMB13CBSA'),
                            ('Oklahoma City, OK','OMB13CBSA'),
                            ('Philadelphia-Camden-Wilmington, PA-NJ-DE-MD','OMB13CBSA'),
                            ('Phoenix-Mesa-Scottsdale, AZ', 'OMB13CBSA'),
                            ('Pittsburgh, PA','OMB13CBSA'),
                            ('Portland-Vancouver-Hillsboro, OR-WA','OMB13CBSA'),
                            ('Raleigh, NC','OMB13CBSA'),
                            ('Richmond, VA','OMB13CBSA'),
                            ('Riverside-San Bernardino-Ontario, CA','OMB13CBSA'),
                            ('Rochester, NY','OMB13CBSA'),
                            ('San Antonio-New Braunfels, TX','OMB13CBSA'),
                            ('San Francisco-Oakland-Hayward, CA','OMB13CBSA'),
                            ('San Jose-Sunnyvale-Santa Clara, CA','OMB13CBSA'),
                            ('Seattle-Tacoma-Bellevue, WA','OMB13CBSA'),
                            ('Tampa-St. Petersburg-Clearwater, FL','OMB13CBSA'),
                            ('Seattle-Tacoma-Bellevue, WA','OMB13CBSA'),
                            ('Washington-Arlington-Alexandria, DC-VA-MD-WV','OMB13CBSA'),
                            ('All other metropolitan areas','OMB13CBSA'),
                            ('Not in a metropolitan area','OMB13CBSA')])

income=widgets.Dropdown(value='HINCP_BIN',style=style, description="Income",disabled=False,
                  options=[('(0.0, 10000.0] ','HINCP_BIN'),
                       ('(10000.0, 20000.0]', 'HINCP_BIN'),
                       ('(20000.0, 30000.0]','HINCP_BIN'),
                       ('(30000.0, 40000.0]','HINCP_BIN'),
                       ('(40000.0, 50000.0]','HINCP_BIN'),
                       ('(50000.0, 60000.0]', 'HINCP_BIN'),
                       ('(60000.0, 70000.0]','HINCP_BIN'),
                       ('(70000.0, 80000.0]',  'HINCP_BIN' ),
                       ('(80000.0, 90000.0]', 'HINCP_BIN'),
                       ('(90000.0, 100000.0]', 'HINCP_BIN')])


In [37]:
left_box=VBox([age,income,number_of_ppl,region])
right_box=VBox([bedrooms,unitsize,utilamt])
ui=widgets.HBox([left_box, right_box])
output=widgets.interactive_output(load_model,{'HHAGE':age,'HINCP_BIN':income,'NUMPEOPLE':number_of_ppl, 'UTILAMT':utilamt,'BEDROOMS':bedrooms,'OMB13CBSA':region, 'UNITSIZE':unitsize})
output.layout.height="400px"
output.layout.width='800px'
box=widgets.VBox([ui,output])
display(box)
